In [1]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA
prob = om.Problem()
prob.model = SellarMDA()
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-8)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()
prob['y2'] = 3.4

In [2]:
import ipywidgets as widgets
from openmdao.core.component import Component
from ipytree import Tree, Node
from ipywidgets import Label, FloatText, HBox, VBox, Output, Layout, Button
import numpy as np

In [3]:
output = Output(layout={'border': '1px solid black', 'width':'60%'})

class SetValuesUI(object):
    def __init__(self, prob, vars_to_set=None):
        """
        Initialize attributes.
        """
        self._prob = prob
        self._vars_to_set = vars_to_set
        self._value_widget_box = None
        self._tree = None
        self.ui_widget = None
        
    def setup(self):
        self._tree = Tree(stripes=True, multiple_selection=True)
        self._value_widget_box = VBox([Label("Model Variables"),])
        if self._vars_to_set:
            self.set_vars_from_user_input(self._vars_to_set, self._tree)
        else:
            self.set_vars_from_model(self._prob.model, self._tree)
        self._tree.observe(self.on_selected_change, names='selected_nodes')
        self.ui_widget = HBox([self._tree, self._value_widget_box, output])
#         self.ui_widget = HBox([self._tree, self._value_widget_box])

    def set_vars_from_model(self, sys, node):
        if sys.name == '_auto_ivc':
            return
        name = sys.name if sys.name else 'root'
        new_node = Node(name)
        node.add_node(new_node)

        if isinstance(sys, Component):
            inputs = list(sys._var_allprocs_prom2abs_list['input'].keys())
            new_node.icon = 'plug'
            for input in inputs:
                input_node = Node(input)
                input_node.icon = 'signal'
                new_node.add_node(input_node)
        else:
            new_node.icon = 'envelope-open'
            for s in sys._subsystems_myproc:
                self.set_vars_from_model(s, new_node)    

    def set_vars_from_user_input(self, vars_to_set, node):
        for var_name in vars_to_set:
            self.add_value_widget(var_name)

    @output.capture()
    def add_value_widget(self, var_name):
        print(type(self._prob[var_name]))
        print(self._prob[var_name])
        print(np.isscalar(self._prob[var_name]))
        val = self._prob[var_name]
        if isinstance(val, np.ndarray):
            if val.size > 1:
                return # skip arrays for now
            val = val.item() 
        val_widget = widgets.FloatText(
            value=self._prob[var_name],
            description=var_name,
            disabled=False
        )
        val_widget.observe(self.update_prob_val, 'value')
        remove_button = Button(description="X")
        remove_button._var_name = var_name # so each Button instance know what variable it is associated with
        remove_button.on_click(self.remove_val_widget)
        val_widget.observe(self.update_prob_val, 'value')
        val_and_remove_widget = HBox([val_widget, remove_button])
        self._value_widget_box.children += (val_and_remove_widget,)  


    @output.capture()
    def on_selected_change(self, change):
        change['new'][0].icon_style = 'warning'
        var_name = change['new'][0].name
        if var_name in self.get_widget_var_names():
            change['new'][0].selected = False
            return
        
        self.add_value_widget(var_name)

    @output.capture()
    def remove_val_widget(self, button):
        if button._var_name not in self.get_widget_var_names():
            raise RuntimeError(f"Trying to remove {button._var_name} which should already be gone" )
        # also need to go through self._value_widget_box.children. Each is an HBox with a FloatText widget 
        for box in self._value_widget_box.children[1:]: # skip the first since it is the Label. The rest are HBoxes
            float_text_widget = box.children[0]
            if button._var_name == float_text_widget.description:
                box_to_remove = box
                break
        # Cannot use remove on tuples, which children are so 
        self._value_widget_box.children = tuple(box for box in self._value_widget_box.children if box != box_to_remove)
        box_to_remove.close()
                
    def update_prob_val(self, change):
        self._prob[change['owner'].description] = change['new']
        
    def get_widget_var_names(self):
        var_names = []
        for box in self._value_widget_box.children[1:]:
            float_text_widget = box.children[0]
            var_names.append(float_text_widget.description)
        return var_names

    def display(self):
        return self.ui_widget

In [4]:
ui = SetValuesUI(prob, vars_to_set=['x','z', 'y2'])

In [5]:
ui.setup()

In [6]:
ui.display()

In [193]:
output = Output(layout={'border': '1px solid black', 'width':'70%'})

class SetValuesUI_old(object):
    def __init__(self, prob, vars_to_set=None):
        """
        Initialize attributes.
        """
        self._prob = prob
        self._vars_to_set = vars_to_set
        self._value_widget_box = None
        self._tree = None
        self._val_widgets = [] # the names
        self.ui_widget = None
        
    def setup(self):
        print('in setup')
        self._tree = Tree(stripes=True, multiple_selection=True)
        self._value_widget_box = VBox([Label("Model Variables"),])
        if self._vars_to_set:
            self.set_vars_from_user_input(self._vars_to_set, self._tree)
        else:
            self.set_vars_from_model(self._prob.model, self._tree)
        self._tree.observe(self.on_selected_change, names='selected_nodes')
        self.ui_widget = HBox([self._tree, self._value_widget_box, output])
#         self.ui_widget = HBox([self._tree, self._value_widget_box])

    def set_vars_from_model(self, sys, node):
        if sys.name == '_auto_ivc':
            return
        name = sys.name if sys.name else 'root'
        new_node = Node(name)
        node.add_node(new_node)

        if isinstance(sys, Component):
            inputs = list(sys._var_allprocs_prom2abs_list['input'].keys())
            new_node.icon = 'plug'
            for input in inputs:
                input_node = Node(input)
                input_node.icon = 'signal'
                new_node.add_node(input_node)
        else:
            new_node.icon = 'envelope-open'
            for s in sys._subsystems_myproc:
                self.set_vars_from_model(s, new_node)    

    def set_vars_from_user_input(self, vars_to_set, node):
        for var in vars_to_set:
            self._val_widgets.append(var)
            val_widget = widgets.FloatText(
                value=7.5,
                description=var,
                disabled=False
            )
            val_widget.observe(self.update_prob_val, 'value')
            self._value_widget_box.children += (val_widget,)      

    @output.capture()
    def on_selected_change(self, change):
        change['new'][0].icon_style = 'warning'
        var_name = change['new'][0].name
        if var_name in self._val_widgets:
            change['new'][0].selected = False
            return
        self._val_widgets.append(var_name)
        val_widget = widgets.FloatText(
            value=7.5,
            description=var_name,
            disabled=False
        )
        remove_button = Button(description="X")
        remove_button._var_name = var_name
        remove_button.on_click(self.remove_val_widget)
        val_widget.observe(self.update_prob_val, 'value')
        val_and_remove_widget = HBox([val_widget, remove_button])
        self._value_widget_box.children += (val_and_remove_widget,)

    @output.capture()
    def remove_val_widget(self, button):
        if button._var_name not in self._val_widgets:
            raise RuntimeError(f"Trying to remove {button._var_name} which should already be gone" )
        self._val_widgets.remove(button._var_name)
        # also need to go through self._value_widget_box.children. Each is an HBox with a FloatText widget 
        for box in self._value_widget_box.children[1:]: # skip the first since it is the Label. The rest are HBoxes
            print(box)
            float_text_widget = box.children[0]
            if button._var_name == float_text_widget.description:
                box_to_remove = box
                break
        # Cannot use remove on tuples, which children are so 
        self._value_widget_box.children = tuple(box for box in self._value_widget_box.children if box != box_to_remove)
        box_to_remove.close()
                
    def update_prob_val(self, change):
        self._prob[change['owner'].description] = change['new']
        

    def display(self):
        return self.ui_widget